In [2]:
import numpy as np
import torch
import pandas as pd
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb
import scipy.sparse as sps
import numpy as np
import torch
torch.manual_seed(2020)
from torch import nn
import torch.nn.functional as F
from math import sqrt

def sigmoid(x):
    return 1./(1. + np.exp(-x))

from dataset import load_data, load_features
from matrix_factorization import MF, MF_N_IPS, MF_N_DR_JL, MF_N_MRDR_JL
from baselines import MF, MF_IPS, MF_ASIPS, MF_SNIPS, MF_DR, MF_DR_JL, MF_MRDR_JL, MF_BaseModel
from models import MLP, MLP_exp, MLP_weibull, MLP_lognormal, MF_IPS_DF, MF_DR_JL_DF

from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU,recall_func, precision_func
from utils import ndcg_func_both, ndcg_func_feature, recall_func_both, recall_func_feature, generate_total_sample
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)


dataset_name = "coat"

if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)
n_train = x_train.shape[0]

train_user_ind = x_train[:, 0].astype('int')
train_item_ind = x_train[:, 1].astype('int')
test_user_ind = x_test[:, 0].astype('int')
test_item_ind = x_test[:, 1].astype('int')

# recover the complete matrix and retrieve the features

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
mf = MF(num_user, num_item, batch_size=128)
mf.to(device)
mf.fit(x_train, y_train, 
    lr=0.05,
    lamb=1e-3,
    tol=1e-4)

x_train_x, u_emb_train, v_emb_train = mf.forward(x_train, True)
x_test_x, u_emb_test, v_emb_test = mf.forward(x_test, True)

# for the feature set of training dataset and test dataset
feature_train = torch.cat([u_emb_train, v_emb_train], axis = 1).detach().cpu()
feature_test = torch.cat([u_emb_test, v_emb_test], axis = 1).detach().cpu()

# the feature of user and item
user_W = mf.W(torch.LongTensor(np.arange(num_user) ).to(device) ).detach().cpu()
item_H = mf.H(torch.LongTensor(np.arange(num_item) ).to(device) ).detach().cpu()

# L = 5, 20%, 19.66%
# L = 6.7, 15%, 15.05%
# L = 3.2, 30%, 30.12%
# L = 9.7, 10%, 10.02%
# L = 3.92, 25%, 24.99%

L = 3.92
sigmaH = 0.1

num_feature = feature_train.shape[1]
identity_p = np.diag(np.ones(num_feature))
mean_p = np.zeros(num_feature)

W_d = np.random.multivariate_normal(mean_p, sigmaH**2*identity_p)
print(W_d)

y_train_mask = np.zeros_like(y_train)
e_train = np.zeros_like(y_train, dtype='float')
d_train = np.zeros_like(y_train, dtype='float') + 1e5

prod = 0
for i in range(n_train):

    ts_i = np.random.uniform(0, L)
    lambda_i = np.exp( np.dot(W_d, feature_train[i, :]) )
    d_i = np.random.exponential(lambda_i)
    e_i = L - ts_i
    if d_i <= e_i:
        y_train_mask[i] = y_train[i]
        d_train[i] = d_i
    else:
        if y_train[i] == 1:
            prod += 1
        y_train_mask[i] = 0
    
    e_train[i] = e_i

print('mask proportion (within the original y_train = 1 )')
print( prod/(sum(y_train) )*100 )

test_pred = mf.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
print('prediction for base model')
print(auc_mf)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

feature_test = feature_test.to(device)

===>Load from coat data set<===
[train] rating ratio: 0.080000
[test]  rating ratio: 0.053333
# user: 290, # item: 300
[MF] epoch:8, xent:37.39917492866516
[-0.04494046 -0.09064704  0.11771674 -0.20835941  0.08493842 -0.07913483
 -0.04008389  0.0490871 ]
mask proportion (within the original y_train = 1 )
24.986195472114854
prediction for base model
0.6076920935476612


In [6]:
# for our model
# for IPS
mf_ips_df_acc = []
for repeat in np.arange(10):
    mf_ips_df = MF_IPS_DF(num_user, num_item, num_feature, batch_size=128)
    mf_ips_df.to(device)
    mf_ips_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, y_ips,lr=0.025, lamb=6e-3, lamb1=4e-2, tol=1e-5)
    test_pred = mf_ips_df.predict(x_test, feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_both(mf_ips_df, x_test, y_test, feature_test)
    recall_res = recall_func_both(mf_ips_df, x_test, y_test, feature_test)

    print(auc)

    mf_ips_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_ips_df_acc = np.array(mf_ips_df_acc)
mf_ips_df_mean = mf_ips_df_acc.mean(0)
mf_ips_df_sd = mf_ips_df_acc.std(0)

print("[MF_IPS_DF] test auc:", mf_ips_df_mean[1], ' sd: ', mf_ips_df_sd[1])
print(mf_ips_df_mean)
print(mf_ips_df_sd)

[MF-IPS-DF] epoch:28, xent:519.4723453521729
0.6798002024499694
[MF-IPS-DF] epoch:37, xent:519.0551261901855
0.6803071006736217
[MF-IPS-DF] epoch:30, xent:519.6696224212646
0.6854933345396814
[MF-IPS-DF] epoch:29, xent:520.8837661743164
0.6899212509830578
[MF-IPS-DF] epoch:28, xent:519.5883502960205
0.6860597768719856
[MF-IPS-DF] epoch:27, xent:520.4755086898804
0.6832069760949736
[MF-IPS-DF] epoch:27, xent:520.5881776809692
0.6854798984502293
[MF-IPS-DF] epoch:25, xent:520.7386665344238
0.6782620118639704
[MF-IPS-DF] epoch:28, xent:519.3867630958557
0.6787601137988446
[MF-IPS-DF] epoch:30, xent:520.6974630355835
0.6869891869445288
[MF_IPS_DF] test auc: 0.6834279852670863  sd:  0.0037639457825530773
[0.23812294 0.68342799 0.59553304 0.66925102 0.40524693 0.6884599 ]
[0.00032772 0.00376395 0.01087502 0.00797329 0.00929618 0.00547514]


In [7]:
# for DR
mf_dr_df_acc = []
for repeat in np.arange(10):
    mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=128)
    mf_dr_df.to(device)
    mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H,  y_ips, lr=0.03, lamb=8e-2, lambv=8e-3, tol=1e-5)
    test_pred = mf_dr_df.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
    recall_res = recall_func(mf_dr_df, x_test, y_test)

    print(auc)

    mf_dr_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_df_acc = np.array(mf_dr_df_acc)
mf_dr_df_mean = mf_dr_df_acc.mean(0)
mf_dr_df_sd = mf_dr_df_acc.std(0)

print("[MF_DR_JL_DF] test auc:", mf_dr_df_mean[1], ' sd: ', mf_dr_df_sd[1])
print(mf_dr_df_mean)
print(mf_dr_df_sd)

[MF-DR-JL-DF] epoch:19, xent:51955897.75
0.7238357773483384
[MF-DR-JL-DF] epoch:25, xent:51953688.1875
0.7243447054847858
[MF-DR-JL-DF] epoch:24, xent:51860143.25
0.7312261098596538
[MF-DR-JL-DF] epoch:22, xent:51917846.0
0.7299939527931214
[MF-DR-JL-DF] epoch:26, xent:51991552.125
0.7145114212560095
[MF-DR-JL-DF] epoch:23, xent:51914872.9375
0.7141106584727788
[MF-DR-JL-DF] epoch:21, xent:52051024.6875
0.7126340419082262
[MF-DR-JL-DF] epoch:22, xent:52011286.9375
0.7203717794950196
[MF-DR-JL-DF] epoch:19, xent:51955229.3125
0.7090337499101038
[MF-DR-JL-DF] epoch:20, xent:51918449.1875
0.7101388924331811
[MF_DR_JL_DF] test auc: 0.7190201088961218  sd:  0.0076589614518389835
[0.33686498 0.71902011 0.6198078  0.68770006 0.43208283 0.71056598]
[0.0005261  0.00765896 0.01662846 0.01235234 0.01500701 0.00812444]


In [8]:
# for weibull
mlp_weibull_acc = []
for repeat in np.arange(10):
    mlp_weibull = MLP_weibull(num_feature, batch_size=128)
    mlp_weibull.to(device)
    mlp_weibull.fit(x_train, y_train_mask, e_train, d_train, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mlp_weibull.predict(feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_feature(mlp_weibull, x_test, y_test, feature_test)
    recall_res = recall_func_feature(mlp_weibull, x_test, y_test, feature_test)

    print(auc)

    mlp_weibull_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mlp_weibull_acc = np.array(mlp_weibull_acc)
mlp_weibull_mean = mlp_weibull_acc.mean(0)
mlp_weibull_sd = mlp_weibull_acc.std(0)

print("[MLP_weibull] test auc:", mlp_weibull_mean[1], ' sd: ', mlp_weibull_sd[1])
print(mlp_weibull_mean)
print(mlp_weibull_sd)

[MLP_weibull] epoch:15, xent:42.51666438579559
0.6326622441633241
[MLP_weibull] epoch:21, xent:42.45467269420624
0.6326624374883522
[MLP_weibull] epoch:15, xent:42.42912805080414
0.6326124629685909
[MLP_weibull] epoch:22, xent:42.48820096254349
0.6326798367408802
[MLP_weibull] epoch:22, xent:42.434920251369476
0.6325964169912595
[MLP_weibull] epoch:18, xent:42.48200082778931
0.6326626308133803
[MLP_weibull] epoch:18, xent:42.452604830265045
0.6326389484974393
[MLP_weibull] epoch:17, xent:42.494261145591736
0.6326856364917229
[MLP_weibull] epoch:21, xent:42.47811549901962
0.6326687205517652
[MLP_weibull] epoch:18, xent:42.463691890239716
0.6326700738269617
[MLP_weibull] test auc: 0.6326539408533677  sd:  2.7612961347012995e-05
[0.23732768 0.63265394 0.52578471 0.60550336 0.35993609 0.64539805]
[3.94379160e-03 2.76129613e-05 1.90942129e-04 1.93470770e-04
 1.85695338e-04 1.11022302e-16]


In [9]:
# for DR
mf_dr_acc = []
for repeat in np.arange(10):
    mf_dr = MF_DR(num_user, num_item, batch_size=128)
    mf_dr.to(device)
    mf_dr.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mf_dr.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr, x_test, y_test)
    recall_res = recall_func(mf_dr, x_test, y_test)

    print(auc)

    mf_dr_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_acc = np.array(mf_dr_acc)
mf_dr_mean = mf_dr_acc.mean(0)
mf_dr_sd = mf_dr_acc.std(0)

print("[MF_DR] test auc:", mf_dr_mean[1], ' sd: ', mf_dr_sd[1])
print(mf_dr_mean)
print(mf_dr_sd)

[MF-DR] epoch:14, xent:57334.51092529297
0.6839302050250587
[MF-DR] epoch:14, xent:57344.67315673828
0.676785781949474
[MF-DR] epoch:13, xent:57369.385681152344
0.6608573462350724
[MF-DR] epoch:14, xent:57281.531799316406
0.6855354793958052
[MF-DR] epoch:17, xent:57386.649963378906
0.6702232672084406
[MF-DR] epoch:18, xent:57357.103271484375
0.663859780583826
[MF-DR] epoch:14, xent:57341.02258300781
0.6735530008297509
[MF-DR] epoch:15, xent:57377.869201660156
0.6909996179897445
[MF-DR] epoch:18, xent:57377.71423339844
0.683141148922909
[MF-DR] epoch:21, xent:57365.63952636719
0.6627490316349343
[MF_DR] test auc: 0.6751634659775014  sd:  0.010074964656029535
[0.2361615  0.67516347 0.57660998 0.65264442 0.38865676 0.68415853]
[0.00040525 0.01007496 0.01333942 0.01333571 0.00984054 0.01366623]
